<a href="https://colab.research.google.com/github/SH0123/DeepLearning/blob/master/mnist_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn

import torchvision.datasets as dsets
import torchvision.transforms as transforms

import torch.nn.init

In [0]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [0]:
print(device)

In [0]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [0]:
#MNIST dataset

mnist_train = dsets.MNIST(root='MNIST_data/',
                           train = True,
                           transform = transforms.ToTensor(),
                           download = True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                           train = False,
                           transform = transforms.ToTensor(),
                           download = True)

In [0]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [0]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.fc = nn.Linear(7*7*64, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, input):
        out = self.layer1(input)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        
        out = self.fc(out)
        return out

In [0]:
model = CNN().to(device)

In [0]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [0]:
#Training
total_batch = len(data_loader)

for epoch in range(training_epochs):
    avg_cost = 0
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)

        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost/total_batch

    print('[Epoch : {}] cost : {}'.format(epoch+1, avg_cost))
print('Learning Finished')

In [0]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test

    accuracy = correct_prediction.float().mean()
    print('Accuray : {}', accuracy.item())
    #about .item() if there is only one value in tensor you can use .item() to get only value